In [ ]:
import psycopg2
import os 
import pandas as pd
import numpy as np

In [ ]:
# function to filter the csv files into a list
def csv_files():
    csv_files=[]
    for file in os.listdir(os.getcwd()):
        if file.endswith(".csv"):
            csv_files.append(file)
    return csv_files


In [ ]:
def configure_dataset_directory(csv_files,dataset_dir):
    # Make directory
    mkdir=f"mkdir {dataset_dir}"
    os.system(mkdir)
    
    # Move csv files to directory
    for csv in csv_files:
        mv_file=f"move {csv} {dataset_dir}"
        os.system(mv_file)
        print(mv_file)

In [ ]:
# main 
dataset_dir=input("Enter the folder name to create?")
csv_files=csv_files()
configure_dataset_directory(csv_files,dataset_dir)
csv_files

In [ ]:
def create_df(csv_files,dataset_dir):
    # Path to csv files
    data_path=os.getcwd()+"\\"+dataset_dir
    # loop throw the csv files and create dataframe
    df={}
    for file in csv_files:
        try:
            df[file]=pd.read_csv(os.path.join(data_path,file))
        except UnicodeDecodeError:
            df[file]=pd.read_csv(os.path.join(data_path,file),encoding="ISO-8859-1")
    return df

In [ ]:
def clean_table_name(filename):
    clean_table_name=filename.lower().replace(" ","_").replace("?","") \
        .replace("-","_").replace( "/" ,"_").replace("\\","_") \
        .replace("%","").replace(")","").replace("(","").replace("$","")
    table_name="{}".format(clean_table_name.split(".")[0])
    return clean_table_name

In [ ]:
def clean_colname(dataframe):
    dataframe.columns=[x.lower().replace(" ","_").replace("?","") \
        .replace("-","_").replace( "/" ,"_").replace("\\","_") \
        .replace("%","").replace(")","").replace("(","").replace("$","")
        for x in dataframe.columns      
        ]
    # replacment dictionary that maps pandas dtypes to sql dtypes
    replacments={
        "object":"varchar",
        "float64":"float",
        "int64":"int"
        }
    col_str=", ".join(["{} {}".format(c,d) for (c,d) in zip(dataframe.columns, \
                                        dataframe.dtypes.replace(replacments))])
    
    return col_str, dataframe.columns

In [ ]:
def upload_to_db(host, dbname, user, password, tbl_name, col_str, file, dataframe, dataframe_columns):

    conn_string = "host=%s dbname=%s user=%s password=%s" % (host, dbname, user, password)
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print('opened database successfully')
    
    #drop table with same name
    cursor.execute("drop table if exists %s;" % (tbl_name))

    #create table
    cursor.execute("create table %s (%s);" % (tbl_name, col_str))
    print('{0} was created successfully'.format(tbl_name)) 
    
    #insert values to table

    #save df to csv
    dataframe.to_csv(file, header=dataframe_columns, index=False, encoding='utf-8')

    #open the csv file, save it as an object
    my_file = open(file)
    print('file opened in memory')
    
    #upload to db
    SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """

    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name, file=my_file)
    print('file copied to db')
    
    cursor.execute("grant select on table %s to public" % tbl_name)
    conn.commit()
    cursor.close()
    print('table {0} imported to db completed'.format(tbl_name))

    return

In [ ]:
#main 

#settings
dataset_dir = 'datasets'

#db settings
host = 'add db ip'
dbname = 'add db name'
user = 'add db username'
password = 'add db pwd'

#configure environment and create main df
csv_files = csv_files()
configure_dataset_directory(csv_files, dataset_dir)
df = create_df(dataset_dir, csv_files)

for k in csv_files:

    #call dataframe
    dataframe = df[k]

    #clean table name
    tbl_name = clean_table_name(k)
    
    #clean column names
    col_str, dataframe.columns = clean_colname(dataframe)
    
    #upload data to db   
    upload_to_db(host, 
                 dbname, 
                 user, 
                 password, 
                 tbl_name, 
                 col_str, 
                 file=k, 
                 dataframe=dataframe, 
                 dataframe_columns=dataframe.columns)